In [15]:
import glob, os
TEST_DATA_FOLDERS = [
    "/n/sd7/trung/csp/data/erica/annotation/test/interview",
    "/n/sd7/trung/csp/data/erica/annotation/test/dating",
    "/n/sd7/trung/csp/data/erica/annotation/test/attentive"
]
TRAIN_DATA_FOLDERS = [
    "/n/sd7/trung/csp/data/erica/annotation/kyoto17/spring/interview",
    "/n/sd7/trung/csp/data/erica/annotation/kyoto17/spring/dating",
    "/n/sd7/trung/csp/data/erica/annotation/kyoto17/spring/attentive",
    #"/n/sd7/trung/csp/data/erica/annotation/kyoto16/interview",
    #"/n/sd7/trung/csp/data/erica/annotation/kyoto16/dating",
    #"/n/sd7/trung/csp/data/erica/annotation/kyoto16/attentive",
    #"/n/sd7/trung/csp/data/erica/annotation/kyoto16/labintro",
]

mode = "train"

from subprocess import call
from struct import unpack, pack
import numpy as np

import IPython
import wave
from pydub import AudioSegment
import re

import MeCab
mt = MeCab.Tagger("-Owakati")
mt.parse('')

OUTPUT_FOLDER = "/n/sd7/trung/csp/data/erica"

In [16]:
words = [s.strip().split(' ', 1) for s in open('/n/rd32/mimura/e2e/data/script/aps_sps/word.id', encoding='eucjp')]
decoder_map_word = {word[0].split('+')[0]: int(word[1]) for word in words}

# words = [s.strip().split(' ', 1) for s in open('/n/sd7/trung/csp/data/erica/word_ids.txt') if s != ""]
# decoder_map_word = {word[0]: int(word[1]) for word in words}
# decoder_map_word = {'<unk>': 0, '<sos>': 1, '<eos>': 2, '<sp>': 3}
print("Vocab Size:", len(decoder_map_word))

TAG_COUNT = 17
def get_tag_id(tag):
    for i, t in enumerate(['pQ', 'cQ', 'sQ', 'checkQ', 'inf', 'off', 'pro', 'sug', 'req', 'ans', 'arg', 'disarg', 'cor', 'acc', 'dec', 'bc', 'oth']):
        if tag.startswith(t): return i
    return None

Vocab Size: 30638


In [17]:
vocab = {}
def get_word_id(word, oov, line, return_word=False):
    '''
    if word == '　': return ['<sp>']
    if mode == "train" or word in decoder_map_word: 
        if word in vocab: vocab[word] += 1
        else: 
            vocab[word] = 1
            return [word]
    else:
        if word in oov: oov[word] += 1
        else: oov[word] = 1
        return [word]
    '''
    
    ret = []
    start = 0
    end = len(word)
    while start < len(word):
        if start >= end: 
            if word in oov: oov[word] += 1
            else: 
                oov[word] = 1
                # print(line, word)
            # return ret + [-1]
            return ['<UNK>'] if return_word else [decoder_map_word['<UNK>']]
        if word[start:end] in decoder_map_word:
            if not return_word: ret.append(decoder_map_word[word[start:end]])
            else: ret.append(word[start:end])
            start = end
            end = len(word)
        else:
            end -= 1
    return ret

def split_words(s, oov, return_word=False):
    s = " ".join(s)
    s.replace("　", " ")
    #s = s.replace(' <', '<')
    #s = s.replace(' ', ' <sp> ')
    #s = s.replace('<', ' <')
    s = s.split(' ')
    ret = []
    for ss in s:
        ss = ss.replace(' ', '')
        if ss and ss[0] =='<':
            ret.append(ss if return_word else decoder_map_word['<sp>'])
            continue
        if ss == '': continue
        tokens = mt.parse(ss).strip()
        tokens = tokens.split(' ')
        for t in tokens:
            ret += get_word_id(t, oov, s, return_word)
    return ret
    
def preproc(line):
    for c in ['I', 'L', 'F', 'D', 'N']:
        line = re.sub(r'\(' + c + '([^\)]*)\)', r'\1', line)
    line = re.sub(r'\(\?([^\)]*)\)', r'\1', line)
    for c in list('`"「」') + ['(L', 'L)', '(F', ' L']:
        line = line.replace(c, '')
    line = re.sub(r'%.*', r'', line)
        
    line = line.replace('{LAUGH}', '')
    line = line.replace('{LAuGH}', '')
    line = line.replace('{COUGH}', '')
    line = re.sub(r'\(P [0-9]*\)', r'', line)
    
    k = line.find('/')
    if k != -1:
        tags = [line[k + 1:].split(';')[0]]
        line = line[:k]
    else: tags = []
        
    env = None
    for i in range(len(tags)):
        if tags[i].find('/') >= 0:
            env, tags[i] = tags[i].split('/', maxsplit=1)
    # tags = re.findall(r'\/(\w*)', line)
    line = re.sub(r'\/\w*', '', line)
    # if len(tags) > 0: line += "<" + tags[0] + ">"
    # if len(tags) > 1: print(tags)
    # print(line)
    return line.strip(), tags, env

In [18]:
filepaths = []
targets = []

targets = []
oov = {}
for DATA_FOLDER in TEST_DATA_FOLDERS if mode == "test" else TRAIN_DATA_FOLDERS:
    for file in glob.glob(os.path.join(DATA_FOLDER, "*.txt")):
        is_start = True
        date, id, obj = os.path.basename(file)[:-4].split('_')[:3]
        longid = date + "_" + id
        year = date[:4]
        
        # if longid != "20171128_03": continue
        
        encoding = 'shift-jis'
        if longid in ['20161205_03', '20161209_05', '20161215_04', '20161208_03', '20161205_07', '20161212_01', '20161209_04', 
                      '20161209_03', '20161208_01', '20161212_03', '20161208_02', '20161207_05', '20161215_03', '20161207_04',
                     '20161205_04']: encoding = 'utf-8'
        
        lines = open(file, encoding=encoding).read().split('\n')
        # print(lines)

        k = 0
        while k < len(lines):
            line = lines[k].strip()
            if longid == '20161209_05':
                start, end = line.split(' ')[1:3]
            else:
                start, end = line.split(' ')[1].split('-')
            start, end = float(start) * 1000, float(end) * 1000
            utt_id = line.split(' ')[0]

            output_wav = os.path.join(OUTPUT_FOLDER, "wav", longid, "%s_%s.wav" % (utt_id, obj))
            output_htk = os.path.join(OUTPUT_FOLDER, "htk", longid, "%s_%s.htk" % (utt_id, obj))
            output_npy = os.path.join(OUTPUT_FOLDER, "npy", longid, "%s_%s.npy" % (utt_id, obj))
            
            if line[-2:] == obj + ':':
                k += 1
                s = []
                tags = []
                envs = set()
                while k < len(lines):
                    line = lines[k].strip()
                    if line[-2:] == obj + ':': break

                    line, tag, env = preproc(line)
                    if env: envs.add(env)
                    if tag: tags += tag
                    if line != '': s.append(line)
                    # if len(tag) > 0: s.append("<" + tag[0] + ">")
                    k += 1
                    
                dtag = ['0'] * TAG_COUNT
                for tag in tags:
                    id = get_tag_id(tag)
                    if id is not None: dtag[id] = '1'
                
                s = split_words(s, oov, False)
                tags = '_'.join([';'.join(tag) for tag in tags])
                
                if len(s) >= 2 and float(end) - float(start) < 5000:
                    #targets.append(' '.join([str(k) for k in s]))
                    targets.append((longid, start, end, obj, ' '.join(dtag), ' '.join([str(k) for k in s]), is_start))
                    is_start = False
                    #targets.append("%s %07d %07d %s %s %s %s" % (longid, start, end, utt_id, obj, ''.join(list(envs)) or '_', s))
                    filepaths.append(output_npy)
            else: k += 1

ls = list(zip(targets, filepaths))
ls.sort()
targets, filepaths = zip(*ls)

In [19]:
print("Utterences:", len(filepaths))

Utterences: 10032


In [20]:
_vocab = { word: vocab[word] for word in vocab } if mode == "train" else vocab
import operator
sorted_words = sorted(_vocab.items(), key=operator.itemgetter(1), reverse=True)
print("Vocab Size:", len(sorted_words))
print(sorted_words)

print("OOV:", len(oov))
sorted_oov = sorted(oov.items(), key=operator.itemgetter(1), reverse=True)
print(sorted_oov)

if mode == "train":
    with open(os.path.join(OUTPUT_FOLDER, 'word_ids.txt'), 'w') as f:
        f.write('\n'.join(['<unk> 0', '<sos> 1', '<eos> 2', '<sp> 4\n']))
        for i, word in enumerate(sorted_words): f.write('%s %d\n' % (word[0], i + 4))

Vocab Size: 0
[]
OOV: 248
[('ー', 590), ('致し', 13), ('ゾンビ', 11), ('ヤ', 9), ('らっしゃる', 9), ('高槻', 9), ('っ', 8), ('なくっ', 8), ('友禅', 8), ('等持院', 8), ('らっしゃい', 7), ('ご存知', 7), ('ガ', 6), ('かかっ', 6), ('烏丸', 6), ('下賀茂', 6), ('かなっ', 6), ('ヨ', 6), ('といった', 5), ('ネ', 5), ('上賀茂', 5), ('餃子', 5), ('。', 4), ('＋', 4), ('奴', 4), ('ジャガイモ', 4), ('リョ', 4), ('橿原', 4), ('メットガラ', 4), ('サムギョプサル', 4), ('ゅう', 4), ('就活', 3), ('ご覧', 3), ('ズ', 3), ('おしゃれ', 3), ('糺', 3), ('断', 3), ('ボーカロイド', 3), ('００７', 3), ('持', 3), ('ヒマラヤスギ', 3), ('日向', 3), ('試薬', 2), ('斎藤', 2), ('萌', 2), ('チョット', 2), ('露木', 2), ('ソウデスネ', 2), ('(', 2), ('惹か', 2), ('ガン', 2), ('採れ', 2), ('ジャ', 2), ('鋼', 2), ('錬金術', 2), ('紋', 2), ('\u3000', 2), ('キョ', 2), ('ハッ', 2), ('橿原神宮', 2), ('混ん', 2), ('ソウネ', 2), ('ショーン・オブ・ザ・デッド', 2), ('オヤ', 2), ('信也', 2), ('匂い', 2), ('チュ', 2), ('智也', 2), ('コンピュータ', 2), ('１', 2), ('ガチ', 2), ('シネマ', 2), ('ラ・ラ・ランド', 2), ('オッケー', 2), ('ビッ', 2), ('ホッ', 2), ('祭典', 2), ('キョウ', 2), ('漬物', 2), ('チュパー', 2), ('タブレット', 2), ('濱田', 2), ('庄司'

In [21]:
#print(filepaths)
#print('\n'.join([t for i, t in enumerate(targets)]))
#with open('/n/sd7/trung/csp/data/erica/inputs.txt', 'w') as f:
#    f.write('\n'.join(filepaths))
    
#with open('/n/sd7/trung/csp/data/erica/targets.txt', 'w') as f:
#    f.write('\n'.join(t[4] for t in targets))
    
with open('/n/sd7/trung/csp/data/erica/%s_notag.txt' % mode, 'w') as f:
    f.write('\n'.join([path + ' ' + '2 ' + target[5] + ' 1' for path, target in zip(filepaths, targets)]))
    
with open('/n/sd7/trung/csp/data/erica/%s_notag_context.txt' % mode, 'w') as f:
    for i in range(len(filepaths)):
        if targets[i][6]:
            f.write('%s\t%s %s %s\t2 %s 1' % (filepaths[i], '0', ' '.join('0' * (TAG_COUNT)), targets[i][4], targets[i][5]))
        else:
            f.write('%s\t%s %s %s\t2 %s 1' % (filepaths[i], ('0' if targets[i - 1][3] == targets[i][3] else '1'), targets[i - 1][4], targets[i][4], targets[i][5]))
        if i != len(filepaths) - 1: f.write('\n')

In [24]:
#wav[test[0]:test[1]].export("/n/sd7/trung/test.wav", format='wav')
#IPython.display.Audio("/n/sd7/trung/test.wav", autoplay=True)

In [ ]:
#wavfile = "/n/sd7/trung/data/erica/htk/20171130_07/0041_U.htk"
#wavfile = wavfile.replace('htk', 'wav')
#feat = get_features(wavfile)
#print(feat)
#IPython.display.Audio(wavfile, autoplay=True)